## Initial configuration

In [ ]:
!pip install -q -U google-generativeai

In [12]:
import google.generativeai as genai
import pandas as pd
import random
import os
import time
from IPython.display import display
from IPython.display import Markdown
import json

In [10]:
#Loading the Google API credentials using a local json
with open("creds.json") as f:
   api_keys = json.load(f)
api_key = api_keys["GOOGLE_API_KEY"]

In [11]:
#Configuring Google Gemini API
genai.configure(api_key=api_key)
model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest")
max_tokens_per_call = 50

In [ ]:
## Creating synthetic data

In [35]:
# Define short-text course details
employee_levels = ["Junior", "Senior", "Manager", "Director"]
employee_roles = ["Programmer", "Accountant", "Tester", "Business Analyist", "Human Resource Manager", "Finance Analyist"]
course_types = ["Beginner", "Intermediate", "Advanced"]
course_topics = [
    "Python Programming",
    "C++ programming",
    "Artificial Intelligence",
    "Business Finance",
    "Human resource transformation",
    "Change management",
    "Procurement",
    "Supply Chain Management",
    "Business analysis",
    "Tableau",
    "Power BI",
    "Selenium",
    "Automated Testing",
]

In [36]:
# Creating a function to generate course data that includes descriptions iteratively without exhausting LLM network limitations
def generate_course_data(batch_size):
  data = []
  course_details_to_process = []

  for batch in range(batch_size):
    # Generate random values
    employee_level = random.choice(employee_levels)
    employee_role = random.choice(employee_roles)
    course_type = random.choice(course_types)
    course_topic = random.choice(course_topics)

    # Generate random course name
    course_name_words = [course_topic, course_type, "Essentials"]
    course_name = " ".join(random.sample(course_name_words, random.randint(1, 3)))

    # Course URL (placeholder, replace with logic to generate URLs based on course details)
    course_url = f"https://www.example.com/course/{course_name.lower().replace(' ', '-')}"

    # Generate random course duration
    course_duration = random.randint(60, 120)  # Between 1 and 2 hours in minutes

    # Append course details for batch processing
    course_details_to_process.append(
        {
            "employee_level": employee_level,
            "employee_role": employee_role,
            "course_type": course_type,
            "course_topic": course_topic,
            "course_name": course_name,
            "course_url": course_url,
            "course_duration": course_duration,
        }
    )

  # Batch API calls to avoid exceeding token limits
  descriptions = {}
  prerequisites = {}
  course_sequences = {}
  learning_objectives = {}
  instructor_infos = {}
  for course_details in course_details_to_process:
    course_topic = course_details["course_topic"]

    # Check if content already generated in this batch
    if course_topic not in descriptions:
      descriptions[course_topic] = generate_description_with_gemini(
          course_details["course_type"], course_topic, max_tokens_per_call
      )
    if course_topic not in prerequisites:
      prerequisites[course_topic] = generate_content_with_gemini(
          f"List some prerequisites for a {course_details['course_type']} course on {course_topic}",
          max_tokens_per_call
      )
    if course_topic not in course_sequences:
      course_sequences[course_topic] = generate_content_with_gemini(
          f"Describe the recommended course sequence for a {course_details['course_type']} course on {course_topic}",
          max_tokens_per_call
      )
    if course_topic not in learning_objectives:
      learning_objectives[course_topic] = generate_content_with_gemini(
          f"List some learning objectives for a {course_details['course_type']} course on {course_topic}",
          max_tokens_per_call
      )
    if course_topic not in instructor_infos:
      instructor_infos[course_topic] = generate_content_with_gemini(
          f"Describe the instructor information for a {course_details['course_type']} course on {course_topic}",
          max_tokens_per_call
      )

   # Populate data with generated content
  for course_details in course_details_to_process:
    course_topic = course_details["course_topic"]
    data.append(
        {
            "Employee Level": course_details["employee_level"],
            "Employee Role": course_details["employee_role"],
            "Course Type": course_details["course_type"],
            "Course Topic": course_topic,
            "Course Name": course_details["course_name"],
            "Course URL": course_details["course_url"],
            "Course Description": descriptions.get(course_topic),
            "Course Prerequisites": prerequisites.get(course_topic),
            "Course Sequence": course_sequences.get(course_topic),
            "Course Duration": f"{course_details['course_duration']} minutes",
            "Learning Objectives": learning_objectives.get(course_topic),
            "Instructor Information": instructor_infos.get(course_topic),
        }
    )

  return pd.DataFrame(data)

In [40]:
#Now create the functions to generate the descriptions with Googlge Gemini without exhausting the interval limits
def generate_description_with_gemini(course_type, course_topic, max_tokens=100):
  prompt = f"Write a {max_tokens}-word description for a {course_type} course on {course_topic}."
  response = model.generate_content(prompt)
  time.sleep(5)
  return response.text 


def generate_content_with_gemini(prompt, max_tokens=50):
  prompt = f"{prompt} with a word limit of {max_tokens} ."
  response = model.generate_content(prompt)
  time.sleep(5)
  return response.text   

In [38]:
#Finally, this function will generate the rows in the dataset
def create_data(rows):
  # Define batch size and output file  
  batch_size = 10
  output_file = "synthetic_course_data.csv"

  # Generate course data in batches
  data = pd.DataFrame()
  for i in range(0, rows, batch_size):
    batch_data = generate_course_data(batch_size)
    data = pd.concat([data, batch_data], ignore_index=True)

  # Save data to CSV
  data.to_csv(output_file, index=False)

  print(f"Synthetic course data generated and saved to {output_file}")

In [ ]:
# In this dataset, we will generate 100 synthetic data rows
create_data(100)

## Query LLM for the correct learning path

In [43]:
df = pd.read_csv("synthetic_course_data.csv")

In [44]:
tokencount = model.count_tokens(f"{df}")

In [45]:
tokencount

total_tokens: 1277

In [52]:
def get_learning_plan(query):
    prompt = f"Here is a CSV file containing the course data that the L and D team offers: {df} Can you suggest a learning path for {query} based only on in the course data that the L & D team offers."
    response = model.generate_content(prompt)
    return Markdown(response.text)

In [53]:
get_learning_plan("junior python programmer looking to become a finance manager")

Here's a potential learning path for a junior Python programmer looking to become a finance manager, based on the provided course data:

**1. Foundational Skills:**

* **Artificial Intelligence (Beginner):** This course provides an introduction to AI concepts, which is increasingly relevant in finance. This can help understand how AI is changing the industry and provide a basis for further exploration.
* **Automated Testing (Beginner):** While not directly finance-related, this course develops programming skills, particularly in Python. These skills are essential for automating tasks and building financial models.
* **Essentials Business Finance (Intermediate):** A core course to grasp fundamental financial concepts. This course will equip the learner with essential knowledge about financial statements, ratios, valuation, etc. 

**2. Finance-Specific Skills:**

* **C++ Programming (Advanced):**  Though not directly Python, this course builds advanced programming skills that are often required in complex financial applications and systems.
* **Change Management (Advanced):**  Understanding the complexities of implementing financial change is crucial for a finance manager. This course provides insights into leading and managing financial transformation. 
* **Essentials HR Transformation (Beginner):**  While not strictly finance, HR skills are essential for effective management. This course provides a foundation for understanding HR processes and how they relate to financial goals. 

**3.  Optional Courses (Based on career interest):**

* **Business Analysis (Intermediate):** This course would help develop skills for understanding business needs and translating them into financial solutions.
* **Selenium (Advanced):**  For those interested in finance technology, this course could be valuable for building automation tools for financial analysis and reporting. 

**Important Considerations:**

* **This is a suggested path; the ideal route may vary based on individual experience and career aspirations.**
* **The provided course data might not cover all relevant topics for finance management.** 
* **Completing these courses will not guarantee a finance management position, but it will provide a solid foundation of knowledge and skills.**

**Beyond Courses:**

* **Practical Experience:**  Apply your learning through projects or internships in finance, even in entry-level roles. 
* **Mentorship:** Seek guidance from experienced finance professionals to gain insights and network.
* **Certification:** Consider finance-related certifications like the Certified Financial Analyst (CFA) or Chartered Financial Consultant (ChFC) for career advancement. 


In [54]:
get_learning_plan("HR analyst trying to become a change management professional")

Here's a suggested learning path for an HR Analyst aiming to become a Change Management professional based on the provided course data:

**1. Foundational HR Knowledge (Optional):**

* **Course:** "Human resource transformation" (Course Type: Beginner)
* **Rationale:**  This course provides a strong base in HR functions, crucial for understanding the context of change within organizations.

**2. Business Analysis Fundamentals:**

* **Course:** "Business analysis" (Course Type: Intermediate) 
* **Rationale:** Change management often requires analyzing existing processes and identifying areas for improvement. This course develops the skills needed for this step.

**3. Change Management Core:**

* **Course:** "Change management" (Course Type: Advanced)
* **Rationale:** This is the cornerstone of your path. It provides an in-depth understanding of change management principles, frameworks, and practical application.

**4. Data Visualization Skills (Optional):**

* **Course:** "Beginner Essentials Tableau" (Course Type: Beginner) 
* **Rationale:** While not strictly change management, data visualization tools like Tableau can help you communicate change impacts and track progress effectively.

**5. Technology Skills (Optional):**

* **Course:** "Automated Testing Beginner Essentials" (Course Type: Beginner)
* **Rationale:**  Understanding basic automation and testing principles can be helpful for analyzing the impact of change on systems and processes.

**Explanation:**

* **Progression:** This learning path starts with HR fundamentals (optional), builds on business analysis skills, and culminates in the advanced change management course. 
* **Optional Courses:** The foundational HR and optional technology courses can be taken if you feel they are necessary to strengthen your skillset.
* **Prioritize Advanced:** The "Change management" course should be a priority as it provides the core knowledge you need for this career transition.
* **Focus on Practical:** Consider the "Change Management" course description, and look for learning objectives that highlight practical application and real-world examples.

**Additional Recommendations:**

* **Networking:** Attending webinars, workshops, or conferences focused on change management can provide valuable insights and opportunities to connect with professionals in the field.
* **Mentorship:**  Seek out a mentor or coach within your organization who has experience in change management. Their guidance can be invaluable.

Remember, this is just a suggested path. You can adjust it based on your existing knowledge, career goals, and the specific needs of your organization. 
